In [ ]:
import sys
sys.path.append("../")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

# Import des classes et fonctions depuis src
from src import (
    SERIES_DIR, TRAIN_CSV, TRAIN_LOCALIZERS_CSV, OUTPUT_DIR,
    print_config
)
from src.bricks import EDA, Preprocessor
from src import show_middle_slices

# Afficher la configuration
print("="*60)
print("Configuration de l'environnement")
print("="*60)
print_config()

# Exploration des Données DICOM

Ce notebook utilise la classe **EDA** pour analyser les données locales de manière systématique.

In [ ]:
# Charger les données
df_train = pd.read_csv(TRAIN_CSV)
df_localizers = pd.read_csv(TRAIN_LOCALIZERS_CSV)

print(f"📊 Données chargées:")
print(f"  - Séries totales: {len(df_train)}")
print(f"  - Localisateurs (avec anévrismes): {len(df_localizers)}")

In [ ]:
# Créer l'analyseur EDA
eda = EDA(df_train, df_localizers, SERIES_DIR)

print(f"✅ Analyseur EDA créé")
print(eda)

In [ ]:
# 1️⃣ Analyser les modalités
modality_counts = eda.analyze_modalities()

In [ ]:
# 2️⃣ Analyser la distribution des anévrismes
aneurysm_counts = eda.analyze_aneurysm_distribution()

In [ ]:
# 3️⃣ Analyser les positions anatomiques
position_counts = eda.analyze_positions()

In [ ]:
# 4️⃣ Visualiser les distributions
eda.plot_aneurysm_distribution()

In [ ]:
# 5️⃣ Détecter les séries défectueuses (optionnel - peut être long)
# Commentez cette cellule si vous avez beaucoup de séries

# defective_series = eda.detect_defective_series()
print("⚠️ Détection des séries défectueuses commentée (peut être long)")
print("Décommentez la cellule pour l'exécuter sur vos 20 séries locales")

In [ ]:
# 6️⃣ Analyser le nombre de slices
slice_stats = eda.analyze_slice_counts()

In [ ]:
# 7️⃣ Rapport complet
print("\n" + "="*60)
print("RAPPORT COMPLET")
print("="*60)
eda.generate_report()

In [ ]:
## Visualisation de Séries DICOM

Utilisons le **Preprocessor** pour charger et visualiser des volumes 3D.

In [ ]:
# Créer un preprocessor
preprocessor = Preprocessor(target_spacing=(0.4, 0.4, 0.4))

print("✅ Preprocessor créé")
print(preprocessor)

In [ ]:
# Sélectionner une série au hasard parmi les locales
import os
import glob

available_series = glob.glob(os.path.join(SERIES_DIR, '*'))

if available_series:
    # Prendre la première série disponible
    example_series = available_series[0]
    series_uid = os.path.basename(example_series)
    
    print(f"📁 Série sélectionnée: {series_uid}")
    print(f"📂 Chemin: {example_series}")
    
    # Compter les fichiers DICOM
    dicom_files = glob.glob(os.path.join(example_series, '*.dcm'))
    print(f"🔢 Nombre de slices: {len(dicom_files)}")
else:
    print("❌ Aucune série DICOM trouvée dans:", SERIES_DIR)

In [ ]:
# Charger et préprocesser le volume avec le Preprocessor
if available_series:
    print("🔄 Chargement et preprocessing du volume...")
    volume = preprocessor.process_volume(example_series)
    
    print(f"✅ Volume préprocessé:")
    print(f"  - Shape: {volume.shape}")
    print(f"  - Min: {volume.min():.4f}")
    print(f"  - Max: {volume.max():.4f}")
    print(f"  - Mean: {volume.mean():.4f}")
else:
    print("⚠️ Passez à la cellule suivante")

In [ ]:
# Visualiser les coupes du volume avec show_middle_slices
if available_series:
    print("📊 Visualisation des coupes centrales (axiale, coronale, sagittale):")
    show_middle_slices(volume)
else:
    print("⚠️ Aucun volume à visualiser")

In [ ]:
## Visualisation de Plusieurs Séries

Visualisons les 3 premières séries disponibles localement.

In [ ]:
# Visualiser les 3 premières séries
n_series_to_show = min(3, len(available_series))

for i in range(n_series_to_show):
    series_path = available_series[i]
    series_uid = os.path.basename(series_path)
    
    print(f"\n{'='*60}")
    print(f"Série {i+1}/{n_series_to_show}: {series_uid}")
    print(f"{'='*60}")
    
    try:
        # Preprocessing
        volume = preprocessor.process_volume(series_path)
        print(f"✅ Volume shape: {volume.shape}")
        
        # Visualisation
        show_middle_slices(volume)
        
    except Exception as e:
        print(f"❌ Erreur lors du traitement: {e}")

In [ ]:
## Conclusion

Ce notebook a utilisé les composants modulaires de `src/` :

### ✅ Classe EDA
- Analyse des modalités
- Distribution des anévrismes
- Positions anatomiques
- Détection de séries défectueuses
- Statistiques sur les slices

### ✅ Classe Preprocessor
- Chargement DICOM
- Resampling à espacement cible
- Cropping du fond
- Normalisation

### ✅ Fonctions de Visualisation
- `show_middle_slices()` pour visualiser les coupes 3D

### 🎯 Prochaines étapes

Consultez le notebook [02_dataset_creation.ipynb](02_dataset_creation.ipynb) pour créer un dataset d'entraînement à partir de ces données.